In [2]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from pathlib import Path
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


# Setting Up NN

In [3]:
model = Sequential()
model.add(Conv2D(32,
                 (3, 3),
                 padding = 'same',
                 activation = 'relu',
                 input_shape=(100, 100,3)))

model.add(Conv2D(32, (3, 3), activation = 'relu'))
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same', activation="relu"))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(512, activation="relu"))

model.add(Dropout(0.50))

model.add(Dense(1, activation="sigmoid"))

model.compile(
    loss='binary_crossentropy',
    optimizer='rmsprop', 
    metrics=['accuracy'] 
)

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 100, 100, 32)      896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 98, 98, 32)        9248      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 98, 98, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 49, 49, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 49, 49, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 49, 49, 64)        36928     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 47, 47, 64)       

# Loading Training Data
- We use keras.preprocessing's ImageDataGenerator to create iterators for our dataset.

In [4]:
batch_size = 64
epochs = 1
# create generator
datagen = ImageDataGenerator()
# prepare an iterators for each dataset

test_datagen = ImageDataGenerator(rescale=1./255)
train_datagen = datagen.flow_from_directory('chest_xray/train/',
                                       target_size=(100, 100),
                                       batch_size= batch_size,
                                       class_mode='binary')
val_datagen = datagen.flow_from_directory('chest_xray/val/',
                                     target_size=(100, 100),
                                     batch_size= batch_size,
                                     class_mode='binary')

# confirm the iterator works
batchX, batchy = train_datagen.next()
print('Batch shape=%s, min=%.3f, max=%.3f' % (batchX.shape, batchX.min(), batchX.max()))

Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Batch shape=(64, 100, 100, 3), min=0.000, max=255.000


# Training the Model

In [5]:
model.fit_generator(
        train_datagen,
        steps_per_epoch=5216 // batch_size, 
        epochs=epochs,
        validation_data=val_datagen,
        validation_steps=16 // batch_size)

Epoch 1/1
81/81 [==============================] - 156s 2s/step - loss: 30.8306 - accuracy: 0.7543 - val_loss: 0.5580 - val_accuracy: 0.6250

# Saving Model's Structure and Weights
We save the model in a json file and the weights in the binary h5 format

In [7]:
model_structure=model.to_json()
f = Path("model_structure.json")
f.write_text(model_structure)
model.save_weights('model_weights.h5') 